# ADAPT-VQE

ADAPT-VQE was introduced by [Grimsley et al](https://www.nature.com/articles/s41467-019-10988-2) as a solution to the often impracticably deep, and not necessarily accurate, static VQE circuits. In ansatze like UCCSD, one easily reaches the order of thousands of gates, even for modestly sized molecules. In UCCSD and its generalized UCCGSD, the number of fermionic excitations in the ansatz scales like $\mathcal{O}(N^2M^2)$, and $\mathcal{O}(N^4)$ respectively. Here $N$ refers to the number of spin-orbitals in the problem basis, and $M$ the number of electrons.

In ADAPT-VQE, an ansatz which approximates not UCCSD/UCCGSD, but in fact FCI, is built iteratively. Over a series of cycles, the ansatz circuit is grown to achieve an approximation to FCI with a minimal number of circuit elements. In this way, ADAPT-VQE is a meta-VQE. At each step, a new ansatz is defined, and its parameters optimized according to conventional VQE. As the cycles proceed, the ansatz grows in both complexity and expressibility. This algorithm comes at the expense of a significant increase in measurement overhead. In order to identify the best operator to append to the present ansatz circuit, a large number of measurements are performed to rank the available operators in order of their ability to further reduce the ansatz state energy.

In this notebook, we'll explore the implementation of this algorithm, by taking advantage of the tools available through the qSDK. 

## Implementation Strategy

ADAPT-VQE relies on three principal steps.

1. Construct generator pool $\{P_i\}$, a set of available operators from which we will iteratively construct the ansatz.
2. Rank operators by evaluating the gradient for each operator in the pool $\frac{\partial E^{(k)}}{\partial \theta_i} = \langle \psi_k|[H,P_i]|\psi_k\rangle$. Exit if the largest partial derivative is less than some cutoff.
3. Append $U_k = \mathrm{exp}[i\theta_k P_k]$(index) to the ansatz $U=\prod_k U_k(\theta_k)$, and optimize $\langle H\rangle$ using VQE.
4. Return to step 2.

There are two obvious avenues along which one can be creative in the implementation of ADAPT-VQE--these are the selection of the operator pool $\{P_i\}$, and the ranking strategy. Above, we have specified that we are using the gradient to perform this ranking procedure. To my knowledge this is the only method which has been used previously, and we will continue to do so here. Regarding the choice of operator pool, the original authors used the elements of the UCCGSD, i.e. the single, double fermionic excitations. Subsequent work has focused on qubit-native operators, and we'll follow this strategy as well. Although the UCCGSD allows for high accuracy with a small number of cycles (and consequently very few variational parameters), the circuit remains very deep, as each excitation consists of many multi-qubit Pauli rotations. 

## Hamiltonian-Inspired Qubit-ADAPT
We are going to use the Hamiltonian to establish a set of pool generators to use in our implementation of ADAPT-VQE. For each term in the Hamiltonian which acts on a distinct combination of qubits, we'll add a pool operator which is guaranteed to have a non-zero commutator with respect to H, and therefore a finite gradient. This will give us a fairly compact set of operators in the pool, all of which are likely to be useful in our ansatz construction. To do this, we will scan through all terms in the qubit Hamiltonian. For each, we will identify the Pauli operators associated with electronic excitation (X and Y gates). We discard Z gates from the Pauli string. If the number of Y gates in a string is even, we can use this to construct an excitation pool element that preserves T-symmetry, by flipping one of the other X gates to a Y, or a Y to an X. Below, we'll flip the first X or Y to its partner.

In [ ]:
def get_pool(qubit_hamiltonian, n_qubits):
    """Use Hamiltonian to identify non-commuting Pauli strings to use as operator pool.
    We identify terms with even number of Y-operators, in order to define excitations 
    which preserve T-reversal symmetry. We remove all Z operators, and we flip the first
    X or Y to its partner (i.e. X <> Y).
    Args:
        qubit_hamiltonian (QubitOperator): input Hamiltonian
        n_qubits (int): number of qubits for Hamiltonian
    
    Returns:
        pool_generators (list of QubitOperator): list of generators
    """
    import numpy as np
    from qsdk.toolboxes.operators.operators import QubitOperator

    pauli_lookup = {'Z':1, 'X':2, 'Y':3}
    pauli_reverse_lookup = ['I', 'Z', 'X', 'Y']
    pool_generators, pool_tuples = list(), list()
    indices = list()

    for term in qubit_hamiltonian.terms:
        pauli_string = np.zeros(n_qubits, dtype=int)
        
        #identify all X or Y factors
        for index, action in term:
            if pauli_lookup[action] > 1:
                pauli_string[index] = pauli_lookup[action]

        #only allow one operator acting on a given set of qubits
        action_mask = tuple(pauli_string > 1)
        if action_mask in indices:
            continue

        #only consider terms with even number of Y
        if sum(pauli_string) % 2 == 0 and sum(pauli_string) > 0:
            #identify qubit operator to change X<>Y
            flip_index = np.where(pauli_string > 1)[0][0]
            pauli_string[flip_index] += (-1)**(pauli_string[flip_index] % 2) 

            #update set of used qubit combinations
            indices.append(action_mask)
            #create QubitOperator for the new generator
            operator_tuple = tuple([(index,pauli_reverse_lookup[pauli]) for index, pauli in enumerate(pauli_string) if pauli > 0])
            #We don't use the coefficients directly, so since we need to multiply by 1.j for evaluating gradients, 
            #I'm just instantiating these with that coefficient in place
            pool_generators.append(QubitOperator(operator_tuple, 1.0j))
            pool_tuples.append(operator_tuple)
    
    return pool_generators

To exemplify the generation of an operator pool, we will apply this method to a simple Hamiltonian consisting of a single fermionic excitation and its Hermitian conjugate.

In [ ]:
from qsdk.toolboxes.operators import QubitOperator
qubit_hamiltonian = QubitOperator(((0, 'X'), (1, 'X'), (2, 'Y'), (3, 'Y')), -1.0) \
                  + QubitOperator(((0, 'X'), (1, 'Y'), (2, 'Y'), (3, 'X')), 1.0) \
                  + QubitOperator(((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y')), 1.0) \
                  + QubitOperator(((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'X')), -1.0) 
pool_generators = get_pool(qubit_hamiltonian, n_qubits=4)
print(f'OPERATOR POOL: {pool_generators}')

We have used a qubit Hamiltonian where all operators act on the same qubit indices. This results in just a single generator pool element. As we traverse the `qubit_hamiltonian` in order, the generator corresponds to the first term in the Hamiltonian, with the first X operator switched to a Y operator. So from XXYY, we have YXYY. 

With the pool generators so defined, we will need to prepare an observable enabling us to get expectation values for: $\frac{\partial E^{(k)}}{\partial \theta_i} = i\langle \psi_k|[H,P_i]|\psi_k\rangle$. I'm going to use openfermion's *commutator* method to construct a set of observables associated with my generator pool.

In [ ]:
from qsdk.electronic_structure_solvers.vqe_solver import VQESolver
from pyscf import gto

H4 = [('H', (0, 0, 0)), ('H', (0, 0, 1)), ('H', (0, 0, 2)), ('H', (0, 0, 3))]
mol = gto.Mole()
mol.atom = H4
mol.basis = 'sto-3g'
mol.build()

vqe_uccsd = VQESolver({'molecule': mol})
vqe_uccsd.build()

qubit_hamiltonian = vqe_uccsd.qubit_hamiltonian
n_qubits = vqe_uccsd.ansatz.circuit.width

We can use the same operator from above to have a look at the commutator for the single pool-element we generated.

## Assembling the ADAPT Workflow
With all of these tools prepared, we can proceed to define an ADAPT-VQE method. We'll continue to take advantage of the tools provided by the qSDK to minimize the overhead in prototyping this algorithm, utilizing the built-in `VQESolver` class to formulate the kernel of our workflow. We'll use the `MolecularData` class to construct our generator pool, and then wrap the `VQESolver` in a loop. For each cycle, we identify the best choice of generator, update the ansatz circuit, and run VQE again.

In [ ]:
from qsdk.electronic_structure_solvers import ADAPTSolver

opt_dict = {"molecule": mol, "frozen_orbitals": 0, "pool": get_pool, "pool_args": (qubit_hamiltonian, n_qubits), "tol": 0.01, "max_cycles": 12, "verbose": False}

adapt_solver = ADAPTSolver(opt_dict)
adapt_solver.build()

## ADAPT-VQE In Practice
For the generator pool we've considered here, the smallest non-trivial problem to consider is H$_4$, as H$_2$ has only one generator and ADAPT terminates at FCI accuracy after a single iteration. So we'll apply this here to H$_4$ as a linear chain. We begin by instantiating the molecule object from pyscf's `gto.Mole` class, and feed this into the `do_adapt_vqe` method we defined above.

In [ ]:
adapt_solver.simulate()

After 15 cycles, we force the cycle to terminate. At this point, the largest gradient is below the level of 0.01. We can now compare the results against the predictions of FCI.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from qsdk.toolboxes.molecular_computation.molecular_data import MolecularData
from qsdk.electronic_structure_solvers.fci_solver import FCISolver
solver = FCISolver()
exact = solver.simulate(mol)
errors = np.array(adapt_solver.energies) - exact
fig,ax = plt.subplots(1,1)
ax.plot(errors)
ax.set_xlabel('ADAPT Iteration')
ax.set_ylabel('Error (Ha)')
ax.set_title('qubit-ADAPT: H$_4$')
ax.set_yscale('log')
print(f'Final Error: {errors[-1] :.4E}')

Ok so after 15 cycles, we have an error of 0.14 mHa, within chemical accuracy of FCI. How does this all compare against UCCSD-VQE?

In [ ]:
vqe_uccsd.simulate()

In [ ]:
print(f'ADAPT-VQE ERROR: {adapt_solver.energies[-1] - exact :0.4E} Ha')
print(f'UCCSD-VQE ERROR: {vqe_uccsd.optimal_energy - exact :0.4E} Ha')

From the perspective of energy accuracy, the two have reached very similar results, within a factor of two. The big advantage here however, are in the resources required for this ansatz circuit:

In [ ]:
print(f'ADAPT RESOURCES:\n {adapt_solver.get_resources()}\n')
print(f'UCCSD RESOURCES:\n {vqe_uccsd.get_resources()}')


We have managed here to reduce the total number of gates by a factor of 10, the number of 2-qubit gates by a factor of nearly 15, and variational gates by a factor of 10. 

We note that ADAPT-VQE has run with 2 more variational parameters than UCCSD-VQE. This introduces additional overhead to the classical optimizer, which must find an optimal solution within a higher-dimensional space. However, the number of variational parameters for ADAPT-VQE is directly dependent on the number of cycles over which ADAPT is run. We then have the opportunity to compromise between optimization complexity and ansatz expressivity by limiting the number of cycles, and therefore the number of variational parameters (and gate number and depth) associated with a computation. This flexibility in resource requirements is then an important advantage for the ADAPT-VQE algorithm over static ansatze.

There is one small problem with qubit inspired ansatz. They do not generally respect the symmetry of the problem. We can now examing the operators $\hat{S}^2$, $\hat{S}_Z$ and the $\hat{N}$ operators to determine if any loss of symmetry has occured. 

In [ ]:
from qsdk.toolboxes.ansatz_generator.fermionic_operators import number_operator, spinz_operator, spin2_operator
from qsdk.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping
s2hat = spin2_operator(int(mol.nao))
szhat = spinz_operator(int(mol.nao))
nhat = number_operator(int(mol.nao))
qu_s2hat = fermion_to_qubit_mapping(s2hat, 'jw')
qu_szhat = fermion_to_qubit_mapping(szhat, 'jw')
qu_nhat = fermion_to_qubit_mapping(nhat, 'jw')
print(adapt_solver.vqe_solver.operator_expectation(qu_s2hat))
print(adapt_solver.vqe_solver.operator_expectation(qu_szhat))
print(adapt_solver.vqe_solver.operator_expectation(qu_nhat))
print(vqe_uccsd.operator_expectation(qu_s2hat))
print(vqe_uccsd.operator_expectation(qu_szhat))
print(vqe_uccsd.operator_expectation(qu_nhat))

## Conclusion

In this notebook, we've explored a simple implementation of the ADAPT-VQE algorithm, using tools available through the qSDK. We have demonstrated how we can leverage these tools to reduce the effort involved in prototyping new quantum algorithms. This allows the user to focus exclusively on the specific components relevant to their research objectives, without getting bogged down in re-building standard infrastructure from scratch. 

The ADAPT-VQE algorithm provides the flexibility to explore different choices of generator pools, and ranking criteria. Feel free to try out alternative implementations to get a better sense of what this algorithm can do.

RP Day
